In [44]:
from pyiceberg.catalog import load_catalog

import os

import duckdb
import polars as pl
from dagster import EnvVar
from dotenv import dotenv_values
from pathlib import Path

env_vars = dotenv_values("/Users/melchior/Developer/opendata-stack-platform/dagster-workspace/team_data_platform/data_platform/.env")

In [45]:
env_vars.get("CLIENT_ID")

'f055a8bec7d71101'

In [46]:
conn = duckdb.connect(env_vars.get("DUCKDB_DATABASE"))
conn.sql("SHOW ALL TABLES")

query = """
    SELECT 
        *
        EXCLUDE(_incremental_timestamp, _record_loaded_timestamp, _is_valid)
    FROM silver_taxi_trips_validated
"""

In [47]:

pyarrow_table = conn.execute(query).arrow()

In [57]:
catalog_name = "pyiceberg_catalog"
# Define the properties for the catalog
config_properties = {
    # Core REST properties
    "uri": "http://localhost:8181/api/catalog",
    "type": "rest",
    "credential": f"f055a8bec7d71101:68eef640ca12ca8758f06bde03a2f0cd",
    "scope": "PRINCIPAL_ROLE:pyiceberg_service_account_role",
    "warehouse": catalog_name,
    # S3 properties MUST be prefixed with 's3.'
    "s3.endpoint": "http://127.0.0.1:9000",
    "s3.access-key-id": "minioadmin",
    "s3.secret-access-key": "minioadmin",
    "s3.region": "us-east-1",
    "s3.path-style-access": "true",
    "s3.allow-http": "true"
}


catalog = load_catalog(
    name=catalog_name,
    **config_properties
)

In [51]:
catalog.create_namespace("default")

In [58]:

table = catalog.create_table(
    "default.taxi_dataset",
    schema=pyarrow_table.schema,
)

ServerError: SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.

In [ ]:
pyarrow_table.schema

In [31]:
df = conn.execute(query).df()

In [ ]:
type(df)
